<a href="https://colab.research.google.com/github/tlimaroberto/spark-stuffs/blob/main/grpby_agg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
!apt update
!apt install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-2.3.1/spark-2.3.1-bin-hadoop2.7.tgz
!tar xf spark-2.3.1-bin-hadoop2.7.tgz
!pip install -q findspark

import os

os.environ["JAVA_HOME"]= "/usr/lib/jvm/java-8-openjdk-amd64/"
os.environ["SPARK_HOME"]= "/content/spark-2.3.1-bin-hadoop2.7/"

!ls

import findspark
findspark.init()

import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
spark


Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:6 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Fetched 252 kB in 2s (132 kB/s)
Reading package lists... Done
Building dependency tree       
Readin

In [14]:
df = spark.read.csv("challenge.csv", header = True)

In [15]:
df.show()

+---------------+--------------+-----------------+----------+
|     ip_address|       Country|      Domain Name|Bytes_used|
+---------------+--------------+-----------------+----------+
|  52.81.192.172|         China| odnoklassniki.ru|       463|
| 119.239.207.13|         China|         youtu.be|        51|
|  68.69.217.210|         China|        adobe.com|        10|
|   7.191.21.223|      Bulgaria|     linkedin.com|       853|
|   211.13.10.68|     Indonesia|          hud.gov|        29|
|   239.80.21.97|      Suriname|       smh.com.au|       218|
|106.214.106.233|       Jamaica|    amazonaws.com|        95|
| 127.242.24.138|         China| surveymonkey.com|       123|
|     99.2.6.139|Czech Republic|     geocities.jp|       322|
|   237.54.11.63|         China|       amazon.com|        83|
| 252.141.157.25|         Japan|      cornell.edu|       374|
|185.220.128.248|       Belgium|       weebly.com|       389|
|   151.77.19.45|   Afghanistan|independent.co.uk|       282|
|  9.161

In [22]:
from pyspark.sql.functions import *
df = df.withColumn('is_Mexico', when(df.Country == 'Mexico','yes').otherwise('no'))

In [23]:
df.show()

+---------------+--------------+-----------------+----------+--------+---------+
|     ip_address|       Country|      Domain Name|Bytes_used|is_China|is_Mexico|
+---------------+--------------+-----------------+----------+--------+---------+
|  52.81.192.172|         China| odnoklassniki.ru|       463|      no|       no|
| 119.239.207.13|         China|         youtu.be|        51|      no|       no|
|  68.69.217.210|         China|        adobe.com|        10|      no|       no|
|   7.191.21.223|      Bulgaria|     linkedin.com|       853|      no|       no|
|   211.13.10.68|     Indonesia|          hud.gov|        29|      no|       no|
|   239.80.21.97|      Suriname|       smh.com.au|       218|      no|       no|
|106.214.106.233|       Jamaica|    amazonaws.com|        95|      no|       no|
| 127.242.24.138|         China| surveymonkey.com|       123|      no|       no|
|     99.2.6.139|Czech Republic|     geocities.jp|       322|      no|       no|
|   237.54.11.63|         Ch

In [34]:
import pyspark.sql.functions as sqlfunct
df1 = df.groupby('is_Mexico').agg(sqlfunc.sum(df.Bytes_used).alias('Soma'))

AttributeError: ignored

In [30]:
df1.show()

+---------+--------+
|is_Mexico|    Soma|
+---------+--------+
|       no|508076.0|
|      yes|  6293.0|
+---------+--------+



In [43]:
df2 = df.groupby("Country").agg(sqlfunc.countDistinct(df.ip_address).alias('Total'))
df2.sort(col('Total').desc()).show()

+--------------+-----+
|       Country|Total|
+--------------+-----+
|         China|  172|
|     Indonesia|  114|
|   Philippines|   65|
|        Russia|   56|
|        Brazil|   35|
|        Poland|   31|
|        Sweden|   28|
|         Japan|   25|
|      Portugal|   23|
|Czech Republic|   23|
|        France|   21|
|          Peru|   19|
|      Colombia|   17|
| United States|   15|
|     Argentina|   14|
|       Ukraine|   14|
|        Mexico|   13|
|      Thailand|   12|
|       Nigeria|   11|
|        Canada|   11|
+--------------+-----+
only showing top 20 rows

